In [ ]:
import pandas as pd

df = pd.read_csv('Dataset .csv')
df.columns = df.columns.str.strip()
print("📋 Column Names:\n", list(df.columns))
print("\n🔍 Sample Data:\n", df.head())


📋 Column Names:
 ['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address', 'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Average Cost for two', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Price range', 'Aggregate rating', 'Rating color', 'Rating text', 'Votes']

🔍 Sample Data:
    Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   


In [ ]:
cuisine_col = next((col for col in df.columns if 'cuisin' in col.lower()), None)
price_col = next((col for col in df.columns if 'price' in col.lower()), None)
name_col = next((col for col in df.columns if 'name' in col.lower()), None)
rating_col = next((col for col in df.columns if 'rating' in col.lower()), None)


print(f"\nDetected Columns:\nName: {name_col}\nCuisine: {cuisine_col}\nPrice: {price_col}\nRating: {rating_col}")



Detected Columns:
Name: Restaurant Name
Cuisine: Cuisines
Price: Price range
Rating: Aggregate rating


In [ ]:
df[cuisine_col] = df[cuisine_col].fillna('')
df[price_col] = df[price_col].fillna('')
if rating_col:
    df[rating_col] = df[rating_col].fillna(df[rating_col].mean())


df['combined_features'] = df[cuisine_col].astype(str) + ' ' + df[price_col].astype(str)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english')
feature_matrix = tfidf.fit_transform(df['combined_features'])

cosine_sim = cosine_similarity(feature_matrix, feature_matrix)


In [ ]:
def recommend_restaurants(cuisine_preference, price_preference, top_n=5):
    input_profile = f"{cuisine_preference} {price_preference}"
    input_vec = tfidf.transform([input_profile])
    sim_scores = cosine_similarity(input_vec, feature_matrix).flatten()
    top_indices = sim_scores.argsort()[-top_n:][::-1]

    display_cols = [name_col, cuisine_col, price_col]
    if rating_col:
        display_cols.append(rating_col)

    return df.iloc[top_indices][display_cols]


In [ ]:
print("\n🎯 Recommendations for: 'Italian' cuisine & 'Moderate' price")
results = recommend_restaurants('Italian', 'Moderate')
print(results)



🎯 Recommendations for: 'Italian' cuisine & 'Moderate' price
     Restaurant Name Cuisines  Price range  Aggregate rating
1106   Da Pizza Zone  Italian            2               0.0
31      Villa Tevere  Italian            4               4.1
3603         Foodies  Italian            1               0.0
9344       San Carlo  Italian            3               4.2
3219  28 Capri Italy  Italian            3               3.9
